In [1]:
import pandas as pd

orders = pd.read_csv("cleaned_data/orders_cleaned.csv")
order_items = pd.read_csv("cleaned_data/order_items_cleaned.csv")
sales_target = pd.read_csv("cleaned_data/sales_target_cleaned.csv")


In [2]:
# Create item-level total value

order_items["item_total_value"] = (
    order_items["amount"] * order_items["quantity"]
)


In [3]:
# Aggregate order_items to ORDER level

order_items_agg = (
    order_items
    .groupby("order_id")
    .agg(
        order_value=("item_total_value", "sum"),
        total_items=("quantity", "sum"),
        order_profit=("profit", "sum")
    )
    .reset_index()
)


In [4]:
order_items_agg.head()

,order_id,order_value,total_items,order_profit
0,B-25601,9599.0,19,-1218.0
1,B-25602,15559.0,22,975.0
2,B-25603,9086.0,25,-180.0
3,B-25604,1543.0,11,22.0
4,B-25605,525.0,7,0.0


In [5]:
# Merge order_items_agg with orders table - (Create main order-level dataset)

orders_full = orders.merge(
    order_items_agg,
    on="order_id",
    how="inner"
)


In [6]:
# Create time-based features

orders_full["order_date"] = pd.to_datetime(orders_full["order_date"])

orders_full["year"] = orders_full["order_date"].dt.year
orders_full["month"] = orders_full["order_date"].dt.month
orders_full["order_month"] = orders_full["order_date"].dt.to_period("M")


In [7]:

orders_full.head()

,order_id,order_date,customername,state,city,order_value,total_items,order_profit,year,month,order_month
0,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,9599.0,19,-1218.0,2018,4,2018-04
1,B-25602,2018-04-01,Pearl,Maharashtra,Pune,15559.0,22,975.0,2018,4,2018-04
2,B-25603,2018-04-03,Jahan,Madhya Pradesh,Bhopal,9086.0,25,-180.0,2018,4,2018-04
3,B-25604,2018-04-03,Divsha,Rajasthan,Jaipur,1543.0,11,22.0,2018,4,2018-04
4,B-25605,2018-04-05,Kasheen,West Bengal,Kolkata,525.0,7,0.0,2018,4,2018-04


In [8]:
# Merge order_items with order_date (ONLY required column)

sales_data = order_items.merge(
    orders[["order_id", "order_date"]],
    on="order_id"
)


In [9]:
# Convert date & extract year/month

sales_data["order_date"] = pd.to_datetime(sales_data["order_date"])

sales_data["year"] = sales_data["order_date"].dt.year
sales_data["month"] = sales_data["order_date"].dt.month


In [10]:
# Aggregate sales to CATEGORY + MONTH (CORE STEP)

category_monthly_sales = (
    sales_data
    .groupby(["category", "year", "month"])
    .agg(actual_sales=("amount", "sum"))
    .reset_index()
)


In [11]:
category_monthly_sales.head()

,category,year,month,actual_sales
0,Clothing,2018,4,13478.0
1,Clothing,2018,5,9518.0
2,Clothing,2018,6,8782.0
3,Clothing,2018,7,2981.0
4,Clothing,2018,8,11822.0


In [12]:
# Merge with sales targets 

sales_vs_target = category_monthly_sales.merge(
    sales_target,
    on=["category", "year", "month"],
    how="left"
)


In [13]:
# Create ONE performance metric

sales_vs_target["achievement_pct"] = (
    (sales_vs_target["actual_sales"] / sales_vs_target["target"]) * 100
).round(2)


In [14]:
sales_vs_target.head()


,category,year,month,actual_sales,target_month,target,achievement_pct
0,Clothing,2018,4,13478.0,2018-04-01,12000,112.32
1,Clothing,2018,5,9518.0,2018-05-01,12000,79.32
2,Clothing,2018,6,8782.0,2018-06-01,12000,73.18
3,Clothing,2018,7,2981.0,2018-07-01,14000,21.29
4,Clothing,2018,8,11822.0,2018-08-01,14000,84.44


In [15]:
orders_full.to_csv("cleaned_data/orders_full.csv",index=False)

sales_vs_target.to_csv("cleaned_data/sales_vs_target.csv",index=False)
